In [1]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
from typing import List
import instructor
from pydantic import BaseModel

class Category(BaseModel):
    primary: str
    secondary: str

class ResultDetail(BaseModel):
    exist: str
    category: Category | None = None
    freshness: str
    description: str

class ResultList(BaseModel):
    result: List[ResultDetail] = []
    description: str

import dashscope

gpt_prompt1='''任务简介：
- 识别每张图片中的主要物品，判断其分类并评估新旧程度。

分类标准：
- 服装：大衣、皮衣、半袖、裤子、裙子、内衣裤、秋衣类、毛衣类、工装、校服。
- 家电：空气净化器、厨房家电、家居家电。
- 乐器：电子乐器、琴类。
- 玩具：小件（手办）、大件、毛绒玩具。
- 图书：儿童绘本、课外书、小说、套装书籍、课本。
- 手机：智能手机、功能机。
- 笔记本电脑：品牌电脑、非品牌电脑。

对于没有给出分类的物品，需要你自行判断一级分类（primary）和二级分类（secondary），此时exist设置为"否"。

回答格式（JSON）：
{
    "exist": "是/否",
    "category": {
        "primary": "一级分类",
        "secondary": "二级分类"
    },
    "freshness": "新/旧/中等",
    "description": "具体描述"
}
'''

def describe_multiple(images, prompt=gpt_prompt1):
    response = dashscope.MultiModalConversation.call(
        model="qwen-vl-plus",
        messages=[{
            "role": "user",
            "content": [{"text": prompt},] + 
            [{"image": image} for image in images]
        }]
    )
    return response

In [2]:
urls = [f"https://milo-test.oss-cn-zhangjiakou.aliyuncs.com/hdd/batch1/image{i+1:03}.png" for i in range(241)]

In [3]:
from IPython.display import Image, display, HTML

In [4]:
%%time
ret = describe_multiple(urls[:1])

CPU times: user 40.1 ms, sys: 8.7 ms, total: 48.8 ms
Wall time: 2.92 s


In [5]:
result = ret.output.choices[0].message.content[0]["text"]

In [6]:
text = result.strip().lstrip("```json").rstrip("```").strip()

In [7]:
print(text)

{
    "exist": "是",
    "category": {
        "primary": "鞋子",
        "secondary": "运动鞋"
    },
    "freshness": "旧",
    "description": "一双白色带有蓝色条纹装饰的运动鞋。"
}


In [8]:
from typing import List
from pydantic import BaseModel

class Category(BaseModel):
    primary: str
    secondary: str

class ResultDetail(BaseModel):
    exist: str
    category: Category | None = None
    freshness: str
    description: str

class ResultList(BaseModel):
    result: List[ResultDetail] = []
    description: str = None

import json
data = json.loads(result.strip().lstrip("```json").rstrip("```").strip())
item = ResultDetail(**data)

In [9]:
table = "<table><tr><th>Text</th><th>Image</th></tr>"
for i in range(1):
    table += f"<tr><td><img src='{urls[i]}' width='100'></td><td>{item}</td></tr>"    

In [10]:
display(HTML(table))

Text,Image
,"exist='是' category=Category(primary='鞋子', secondary='运动鞋') freshness='旧' description='一双白色带有蓝色条纹装饰的运动鞋。'"
